# Space Time Cube

This code details how I downloaded the annual 30-Year Normals .bil files for precipitation from PRISM, added them to the map and created the spacetime cube.

In [45]:
import requests
import json
import pprint
import zipfile
import arcpy
import pandas as pd
import os

## Requesting the PRISM Data (.bil)

In [1]:
# Creating the base of the URL
base_url = 'http://services.nacse.org/prism/data/public/normals/4km'

In [47]:
# Creating the parameters for each of the datasets (.bil)
element = {'precipitation':'ppt'}
date = {'yyyy':'2010'}

In [48]:
url_final = base_url + '/' + element['precipitation'] + '/'+ date['yyyy']
print(url_final)

http://services.nacse.org/prism/data/public/normals/4km/ppt/2010


In [49]:
# Requesting zipfile of .bil files
r = requests.get(url_final)

In [50]:
# Checking the response
print(r)

<Response [200]>


## Opening the file and Adding it to the Map

In [53]:
# Opening the file
open('PRISM_ppt_30yr_normal_4kmM2_annual_bil.zip', 'wb').write(r.content)
print('extracting the content...')

PermissionError: [Errno 13] Permission denied: 'PRISM_ppt_30yr_normal_4kmM2_annual_bil.zip'

In [28]:
# Unzipping the bil files to the directory on my computer
with zipfile.ZipFile('PRISM_ppt_30yr_normal_4kmM2_annual_bil.zip', 'r') as zip_ref:
    zip_ref.extractall('C:/Users/ecava/OneDrive/Documents/ArcGIS/Projects/Lab2_pt1_3/bil_files')

In [54]:
# Using Arcpy.mp to add the raster to the map
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Map")[0]

In [55]:
# Adding the Raster to the geodatabase at hand
m.addDataFromPath(r'C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt1_3\bil_files\PRISM_ppt_30yr_normal_4kmM2_annual_bil.bil')

In [56]:
# Resampling raster
arcpy.management.Resample("PRISM_ppt_30yr_normal_4kmM2_annual_bil.bil", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt1_3\Lab2_pt1_3.gdb\resampled_raster", "0.041666666667 0.041666666667", "NEAREST")

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab2_pt1_3\\Lab2_pt1_3.gdb\\resampled_raster'>

## Creating the SpaceTime Cube

In [2]:
# Creating an empty mosaic dataset
arcpy.management.CreateMosaicDataset(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt1_3\Lab2_pt1_3.gdb", "prism_ppt_created", 'GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', None, '', "NONE", None)

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab2_pt1_3\\Lab2_pt1_3.gdb\\prism_ppt_created'>

In [6]:
# Adding the empty raster to a mosaic datset
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt1_3\Lab2_pt1_3.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt1_3\Lab2_pt1_3.gdb"):
    arcpy.management.AddRastersToMosaicDataset(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt1_3\Lab2_pt1_3.gdb\prism_ppt_created", "Raster Dataset", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt1_3", "UPDATE_CELL_SIZES", "UPDATE_BOUNDARY", "NO_OVERVIEWS", None, 0, 1500, None, '', "SUBFOLDERS", "ALLOW_DUPLICATES", "NO_PYRAMIDS", "NO_STATISTICS", "NO_THUMBNAILS", '', "NO_FORCE_SPATIAL_REFERENCE", "NO_STATISTICS", None, "NO_PIXEL_CACHE", r"C:\Users\ecava\AppData\Local\ESRI\rasterproxies\prism_ppt_created")

In [7]:
# Calculating the 'variable' field
arcpy.management.CalculateField(r"prism_ppt_created\Footprint", "Variable", "'ppt'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'prism_ppt_created\\Footprint'>

In [8]:
# Calculating the timestamp field
arcpy.management.CalculateField(r"prism_ppt_created\Footprint", "Timestamp", "DateAdd(Date(1981,0,1), $feature.OBJECTID-1, 'year')", "ARCADE", '', "DATE", "NO_ENFORCE_DOMAINS")

<Result 'prism_ppt_created\\Footprint'>

In [9]:
# Building multidimensional Information in the mosaic of rasters
arcpy.md.BuildMultidimensionalInfo("prism_ppt_created", "Variable", "Timestamp # #", None, "NO_DELETE_MULTIDIMENSIONAL_INFO")

<Result 'prism_ppt_created'>

In [10]:
# Making a multi-dimensional raster layer 
arcpy.md.MakeMultidimensionalRasterLayer("prism_ppt_created", "prism_ppt_created_yearly_MultidimLayer", "ppt", "ALL", None, None, '', '', '', None, '', "-125.020833333333 24.0625000029491 -66.4791666733335 49.9375000000025", "DIMENSIONS")

<Result 'prism_ppt_created_yearly_MultidimLayer'>

In [8]:
# Creating Space Time Cube
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer("prism_ppt_created_MultidimLayer", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt1_3\stc_prism_30yr_ppt.nc", "ZEROS")

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab2_pt1_3\\stc_prism_30yr_ppt.nc'>